In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

In [ ]:
sys.path.append("./")

In [ ]:
os.path.dirname(os.path.abspath(__file__))

In [ ]:
from clients.sfclient import SnowflakeClient
sf_client = SnowflakeClient()

In [ ]:
sf_client.create_database()

In [ ]:
sf_client.delete_users_table()

In [ ]:
sf_client.create_users_table()

In [ ]:
sf_client.insert_users_data()

In [ ]:
sf_client.delete_orders_table()

In [ ]:
sf_client.create_orders_table()

In [ ]:
sf_client.insert_orders_data()

In [ ]:
from clients.bqclient import BigQueryClient

bq_client = BigQueryClient()

In [ ]:
query: str = f"""
    SELECT DISTINCT
    FORMAT_TIMESTAMP('%Y-%m', order_time) AS year_month
    FROM {bq_client.dataset_id}.{bq_client.orders_table_id}
    ORDER BY year_month;
"""
result = bq_client.execute_query(query)
year_months =[row.values()[0] for row in result]

In [ ]:
result.num_results

In [ ]:
pivot_query = f"""
    SELECT u.user_id,
           {', '.join([f"IFNULL(SUM(IF(FORMAT_TIMESTAMP('%Y-%m', o.order_time) = '{ym}', 1, 0)), 0) AS `{ym}`" for ym in year_months])}
    FROM test_db.users u
    LEFT JOIN test_db.orders o
    ON u.user_id = o.user_id
    GROUP BY u.user_id
"""
print(pivot_query)

In [ ]:
pivot_query = f"""
    SELECT u.user_id,
        {', '.join([f"IFNULL(SUM(IF(FORMAT_TIMESTAMP('%Y-%m', o.order_time) = '{ym}', 1, 0)), 0) AS `{ym}`" for ym in year_months])}
    FROM {bq_client.dataset_id}.{bq_client.users_table_id} u
    LEFT JOIN {bq_client.dataset_id}.{bq_client.orders_table_id} o
    ON u.user_id = o.user_id
    GROUP BY u.user_id
    ORDER BY u.user_id;
"""
print(pivot_query)
result = bq_client.execute_query(query=pivot_query)
print(f"bq_list_orders_count_per_user_and_yearmonth: {result.total_rows}")
# assert result.num_results == 10000

In [ ]:
result.total_rows

In [ ]:
result.num_results

In [ ]:
results = bq_client.client.query(pivot_query, project=bq_client.project_id)

In [ ]:
for row in results:
    print(row)

In [ ]:
result.schema

In [ ]:
pivot_query = f"""
    SELECT * FROM (
        SELECT user_id,
               TO_VARCHAR(order_time, 'YYYY-MM') AS year_month
        FROM orders
        LEFT JOIN orders o
        ON u.user_id = o.user_id
    )
    PIVOT (
        COUNT(year_month)
        FOR year_month IN ({', '.join(["'" + ym + "'" for ym in year_months])})
    )
    ORDER BY user_id;
"""

print(pivot_query)

In [ ]:
query = f"""
    SELECT
        u.user_id,
        u.username,
        COUNT(o.order_id) AS total_orders
    FROM
        {bq_client.dataset_id}.{bq_client.users_table_id} u
    LEFT JOIN
        {bq_client.dataset_id}.{bq_client.orders_table_id} o ON u.user_id = o.user_id
    GROUP BY
        u.user_id, u.username
    ORDER BY
        total_orders DESC;
"""

In [ ]:
result = bq_client.execute_query(query)

In [ ]:
for row in result:
    print(row)

In [ ]:
bq_client.truncate_table(bq_client.orders_table_id)

In [ ]:
bq_client.create_dataset()

In [ ]:
bq_client.create_users_table()

In [ ]:
bq_client.insert_users_data()

In [ ]:
bq_client.create_orders_table()

In [ ]:
bq_client.insert_orders_data()

In [ ]:
import pandas as pd

users_df = pd.read_pickle("tests/faker_users.pkl")

In [ ]:
[(row.user_id, row.username, row.email) for row in users_df.itertuples()]

In [ ]:
orders_df = pd.read_pickle(filepath_or_buffer="tests/faker_orders.pkl")

In [ ]:
[(row.order_id, row.user_id, row.product, row.amount, row.order_time) for row in orders_df.itertuples()]